In [1]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
    config = 'configs.config_ILLAE_local'
#     config = 'configs.config_WomenInEngineering_local'
#     config = 'configs.config_Digits_local'    

In [2]:
### initialization ###

import importlib
print('config file is set to {}'.format(config))

imp=importlib.import_module('_imports')
importlib.reload(imp)

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_ILLAE_local


In [3]:
### creating parameters grid for the algorithm hyperparameters ###

experiments_cv=[]

if c.algorithm['name'] == 'svm':
    for kernel in c.algorithm['kernels']:
#         params_svm=imp.copy.deepcopy(c.algorithm['params'])
        params=c.algorithm['params']
        params_svm=dict()
        params_svm['cost'] = imp.copy.deepcopy(params['cost'])    

        if kernel != 'linear':
            params_svm['gamma'] = imp.copy.deepcopy(params['gamma'])
        if kernel == 'poly':
            params_svm['degree'] = imp.copy.deepcopy(params['degree'])
        if kernel == 'poly' or kernel == 'sigmoid':        
            params_svm['coef0'] = imp.copy.deepcopy(params['coef0'])

        keys, values = zip(*params_svm.items())
        experiments = [dict(zip(keys, v)) for v in imp.itertools.product(*values)]
        print('number of experiments with {} kernel = {}'.format(kernel, len(experiments)))
        result_exps=[]

        for experiment in experiments:
            experiment['kernel']=kernel

            for cv in range(c.inner):
                experiment['cv']=cv
                experiments_cv.append(imp.copy.deepcopy(experiment))

experiments_cv_df=imp.pd.DataFrame(experiments_cv)
experiments_cv_df.fillna(value=1, inplace=True)

print(experiments_cv_df.shape)
util.display_df(experiments_cv_df.head(11))

number of experiments with linear kernel = 1
number of experiments with poly kernel = 2
number of experiments with rbf kernel = 1
number of experiments with sigmoid kernel = 1
(25, 6)


,coef0,cost,cv,degree,gamma,kernel
0,1.0,1,0,1.0,1.0,linear
1,1.0,1,1,1.0,1.0,linear
2,1.0,1,2,1.0,1.0,linear
3,1.0,1,3,1.0,1.0,linear
4,1.0,1,4,1.0,1.0,linear
5,10.0,1,0,2.0,0.1,poly
6,10.0,1,1,2.0,0.1,poly
7,10.0,1,2,2.0,0.1,poly
8,10.0,1,3,2.0,0.1,poly
9,10.0,1,4,2.0,0.1,poly


In [4]:
### adding dynamic features parameters to the parameter search ###

for feature in c.select_features['dynamic']:
    for param_name, param_list in c.features['dynamic'][feature]['params'].items():
        param_name='{}_{}'.format(feature, param_name)
        print(param_name, param_list)
        
        experiment_cvs_updated=[]
        for param_val in param_list:
            for experiment in experiments_cv:
                experiment_copy=imp.copy.deepcopy(experiment)
                experiment_copy[param_name]=param_val
                experiment_cvs_updated.append(experiment_copy)
        experiments_cv=experiment_cvs_updated        

experiments_cv_df=imp.pd.DataFrame(experiments_cv)
experiments_cv_df.fillna(value=1, inplace=True)

print(experiments_cv_df.shape)
util.display_df(experiments_cv_df.head(15))

lda_hashtags_no_below [2]
lda_hashtags_no_above [0.6]
lda_hashtags_LDA_topic_cutoff_threshold [0.1]
lda_hashtags_LDA_num_topics [20]
(25, 10)


,coef0,cost,cv,degree,gamma,kernel,lda_hashtags_LDA_num_topics,lda_hashtags_LDA_topic_cutoff_threshold,lda_hashtags_no_above,lda_hashtags_no_below
0,1.0,1,0,1.0,1.0,linear,20,0.1,0.6,2
1,1.0,1,1,1.0,1.0,linear,20,0.1,0.6,2
2,1.0,1,2,1.0,1.0,linear,20,0.1,0.6,2
3,1.0,1,3,1.0,1.0,linear,20,0.1,0.6,2
4,1.0,1,4,1.0,1.0,linear,20,0.1,0.6,2
5,10.0,1,0,2.0,0.1,poly,20,0.1,0.6,2
6,10.0,1,1,2.0,0.1,poly,20,0.1,0.6,2
7,10.0,1,2,2.0,0.1,poly,20,0.1,0.6,2
8,10.0,1,3,2.0,0.1,poly,20,0.1,0.6,2
9,10.0,1,4,2.0,0.1,poly,20,0.1,0.6,2


In [5]:
### saving parameters grid ###

util.mkdir('{}experiments/{}/'.format(c.directory['save'], util.experiment_name(c)))
experiments_cv_df.to_csv('{}experiments/{}/1_parameters.csv'.format(c.directory['save'],util.experiment_name(c)),
                                                                  index=False)

folder /Users/habibkarbasian/Documents/_Courses/_Datasets/_Retweetability/_ILLAE/experiments/ILLAE_test_svm_fffs_age_listed_verified_character_url_media_mentions_reply_hashtag_liwc_lda(lda_hashtags)/ already exists!


In [6]:
### creating parameters grid for dynamic feature parameters ###

experiments_cv=[]
experiment=dict()

for cv in range(c.inner):
    experiment['cv']=cv
    experiments_cv.append(imp.copy.deepcopy(experiment))

experiments_dynamics=[]
for feature in c.select_features['dynamic']:
    experiments_cv_copy=imp.copy.deepcopy(experiments_cv)
    for param_name, param_list in c.features['dynamic'][feature]['params'].items():
        param_name='{}_{}'.format(feature, param_name)
        print(param_name, param_list)
        
        experiment_cvs_updated=[]
        for param_val in param_list:
            for experiment in experiments_cv_copy:
                experiment_copy=imp.copy.deepcopy(experiment)
                experiment_copy[param_name]=param_val
                experiment_copy['name']=feature
                experiment_copy['type']=c.features['dynamic'][feature]['type']                
                experiment_cvs_updated.append(experiment_copy)
        experiments_cv_copy=experiment_cvs_updated
    experiments_dynamics.extend(experiments_cv_copy)

experiments_cv_df=imp.pd.DataFrame(experiments_dynamics)
experiments_cv_df.fillna(value='na', inplace=True)

print(experiments_cv_df.shape)
util.display_df(experiments_cv_df.head(11))

lda_hashtags_no_below [2]
lda_hashtags_no_above [0.6]
lda_hashtags_LDA_topic_cutoff_threshold [0.1]
lda_hashtags_LDA_num_topics [20]
(5, 7)


,cv,lda_hashtags_LDA_num_topics,lda_hashtags_LDA_topic_cutoff_threshold,lda_hashtags_no_above,lda_hashtags_no_below,name,type
0,0,20,0.1,0.6,2,lda_hashtags,lda
1,1,20,0.1,0.6,2,lda_hashtags,lda
2,2,20,0.1,0.6,2,lda_hashtags,lda
3,3,20,0.1,0.6,2,lda_hashtags,lda
4,4,20,0.1,0.6,2,lda_hashtags,lda


In [7]:
### saving dynamic parameters grid ###

experiments_cv_df.to_csv('{}experiments/{}/1_parameters_dynamic_features.csv'.format(c.directory['save'],
                                                                                     util.experiment_name(c)),
                         index=False)